In [1]:
import torch
import torch.nn as nn

In [5]:
a = torch.randn([5,1,10,100,100])

In [62]:
conv1 = nn.Conv3d(in_channels=1,out_channels=64,kernel_size=5,stride=1,padding=2)
conv2 = nn.Conv3d(in_channels=64,out_channels=128,kernel_size=3,stride=(1,2,2),padding=1)
conv3 = nn.Conv3d(in_channels=128,out_channels=128,kernel_size=3,stride=(2,2,2),padding=1)

deconv1 = nn.ConvTranspose3d(in_channels=128,out_channels=128,kernel_size=3,stride=(2,2,2),padding=1,output_padding=1)
deconv2 = nn.ConvTranspose3d(in_channels=128,out_channels=64,kernel_size=(5,4,4),stride=(1,2,2),padding=1,output_padding=0)
deconv3 = nn.ConvTranspose3d(in_channels=64,out_channels=1,kernel_size=5,stride=1,padding=2,output_padding=0)

In [64]:
deconv3(deconv2(deconv1(conv3(conv2(conv1(a)))))).squeeze().shape

torch.Size([5, 12, 100, 100])